# 3.3 线性回归的简洁实现

In [1]:
import torch
from torch import nn
import numpy as np
torch.manual_seed(1)

print(torch.__version__)
torch.set_default_tensor_type('torch.FloatTensor')

1.3.0


In [2]:
help(torch.manual_seed)

Help on function manual_seed in module torch.random:

manual_seed(seed)
    Sets the seed for generating random numbers. Returns a
    `torch.Generator` object.
    
    Args:
        seed (int): The desired seed.



## 3.3.1 生成数据集

In [3]:
num_inputs = 5
num_examples = 2000
true_w = torch.tensor([2, -3.4, 27.4, 4.9, -125.2])
true_b = torch.tensor([106.8])
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)

labels = torch.tensor(np.zeros_like(features[:, 0]), dtype=torch.float)
for i in range(len(true_w)):
    labels += true_w[i] * features[:, i]
labels += true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

## 3.3.2 读取数据

In [4]:
import torch.utils.data as Data

batch_size = 10

# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)

# 把 dataset 放入 DataLoader
data_iter = Data.DataLoader(
    dataset=dataset,      # torch TensorDataset format
    batch_size=batch_size,      # mini batch size
    shuffle=True,               # 要不要打乱数据 (打乱比较好)
    num_workers=8,              # 多线程来读数据
)

In [5]:
features.size(), labels.size()

(torch.Size([2000, 5]), torch.Size([2000]))

In [6]:
# help(Data.TensorDataset)

In [7]:
# help(Data.DataLoader)

In [8]:
for X, y in data_iter:
    print(X, '\n', y)
    break

tensor([[-0.3357,  1.8124, -0.7527, -1.0183,  1.2000],
        [-1.1380, -0.5396,  0.4524,  0.7920, -0.0205],
        [ 0.8881, -0.5873,  0.0812,  0.1208, -0.3787],
        [ 0.0803,  0.1143, -1.7549, -1.6654,  0.7428],
        [ 1.5292, -1.0991,  1.1452,  1.5711, -0.4057],
        [ 0.4851,  1.0806, -0.8622,  1.5168,  1.7333],
        [ 0.1075,  1.6090, -0.3655, -0.1486, -0.3680],
        [-0.1329, -1.0327, -1.4438,  1.1352, -2.6358],
        [-0.7489, -0.2435, -0.3523,  1.1584, -2.1633],
        [-0.4431,  1.2357, -0.2896,  0.0471,  0.3796]]) 
 tensor([ -75.8907,  125.2059,  160.8081,  -42.6653,  203.4595, -129.0935,
         136.8881,  406.0451,  373.0134,   46.4836])


## 3.3.3 定义模型

In [9]:
# class LinearNet(nn.Module):
#     def __init__(self, n_feature):
#         super(LinearNet, self).__init__()
#         # out_features is set as 1
#         self.linear = nn.Linear(n_feature, 1)

#     def forward(self, x):
#         y = self.linear(x)
#         return y
    
# net = LinearNet(num_inputs)
# print(net) # 使用print可以打印出网络的结构

In [10]:
# help(nn.Linear)

In [11]:
# # 写法一
# net = nn.Sequential(
#     nn.Linear(num_inputs, 1)
#     # 此处还可以传入其他层
#     )

# 写法二
net = nn.Sequential()
net.add_module('linear', nn.Linear(num_inputs, 1))
# net.add_module ......

# # 写法三
# from collections import OrderedDict
# net = nn.Sequential(OrderedDict([
#           ('linear', nn.Linear(num_inputs, 1))
#           # ......
#         ]))

# print(net)
# print(net[0])

In [12]:
# for param in net.parameters():
#     print(param)

## 3.3.4 初始化模型参数

In [13]:
from torch.nn import init

init.normal_(net[0].weight, mean=0.0, std=0.01);
init.constant_(net[0].bias, val=0.0);  # 也可以直接修改bias的data: net[0].bias.data.fill_(0)

# init.normal_(net.linear.weight, mean=0.0, std=0.01)
# init.constant_(net.linear.bias, val=0.0)  # 也可以直接修改bias的data: net[0].bias.data.fill_(0)

In [14]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.0017,  0.0145, -0.0032, -0.0083,  0.0052]], requires_grad=True)
Parameter containing:
tensor([0.], requires_grad=True)


## 3.3.5 定义损失函数

In [15]:
loss = nn.MSELoss()

## 3.3.6 定义优化算法

In [16]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.01
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [17]:
# help(optim.SGD)

In [18]:
# 为不同子网络设置不同的学习率
# optimizer =optim.SGD([
#                 # 如果对某个参数不指定学习率，就使用最外层的默认学习率
#                 {'params': net.subnet1.parameters()}, # lr=0.03
#                 {'params': net.subnet2.parameters(), 'lr': 0.01}
#             ], lr=0.03)

In [19]:
# # 调整学习率
# for param_group in optimizer.param_groups:
#     param_group['lr'] *= 0.1 # 学习率为之前的0.1倍

## 3.3.7 训练模型

In [20]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        optimizer.zero_grad() # 梯度清零，等价于net.zero_grad()
        output = net(X)
        l = loss(output, y.view(-1, 1))
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))

epoch 1, loss: 10.173575
epoch 2, loss: 0.005704
epoch 3, loss: 0.000066


In [21]:
# help(optimizer.step)

In [22]:
# help(l.backward)

In [23]:
dense = net[0]
print(true_w)
print(dense.weight.data)
print(true_b)
print(dense.bias.data)

# dense = net
# print(true_w, dense.linear.weight.data)
# print(true_b, dense.linear.bias.data)

tensor([   2.0000,   -3.4000,   27.4000,    4.9000, -125.2000])
tensor([[   1.9999,   -3.3997,   27.3998,    4.9002, -125.1992]])
tensor([106.8000])
tensor([106.7997])
